In [1]:
import numpy as np
from ExtendedKalmanFilter import EKF

# Extended Kalman Filter

See EKF.py for implementation
- create an EKF object
- call filter() with the new measured position and the speed to estimate the new position. 

In [3]:
initial_pos = np.array([0,0,0])
initial_speed = np.array([0,0])
measurements = np.array([
    [0,0,0],
    [1,0,90]
])

speed = np.array([
    [1, 0],
    [0, 90]
])
ekf = EKF(1, initial_pos, initial_speed)
print(ekf.filter(measurements[0], speed[0]))
ekf.filter(measurements[1], speed[1])

State Estimate Before EKF=[1.01  0.01  0.003]
Observation=[0 0 0]
zeros
State Estimate After EKF=[ 0.815 -0.004 -0.005]
(0.8152459016393442, -0.0044262295081967246, -0.00475409836065574)
State Estimate Before EKF=[ 0.825  0.006 89.998]
Observation=[ 1  0 90]
State Estimate After EKF=[ 0.894 -0.044 89.973]


(0.8938983050847458, -0.043954802259887, 89.97318079096046)

In [5]:
x,y,o = ekf.filter(measurements[0], us[0])



State Estimate Before EKF=[ 0.539  0.866 89.972]
Observation=[1 0 0]
State Estimate After EKF=[ 0.78  0.29 34.53]


In [6]:
x

0.7796926905865056